# Exploration des données pour conseiller nos clients sur l'achat de nos véhicules disponibles sur notre plateforme AutoPredict


Vous pourrez, ci-dessous, consulter nos statistiques pour savoir quels types de véhicules nous pourrions vous orienter en fonction de vos besoins.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go


In [2]:
import pandas as pd
import plotly.express as px
import plotly.offline as pyo

# Activation du mode notebook (important pour VS Code)
pyo.init_notebook_mode(connected=True)

# Chargement du fichier CSV
df = pd.read_csv("voiture.csv")

# Nettoyage : colonnes nécessaires et valeurs manquantes
df_clean = df[['MSRP', 'Transmission Type']].dropna()

# Suppression des transmissions UNKNOWN
df_clean = df_clean[df_clean['Transmission Type'] != 'UNKNOWN']

# Création de quantiles de prix
df_clean['MSRP Quantile'] = pd.qcut(df_clean['MSRP'], q=5)

# Conversion en string pour Plotly
df_clean['MSRP Quantile'] = df_clean['MSRP Quantile'].astype(str)

# Tri explicite des catégories dans l'ordre croissant
ordered_quantiles = sorted(df_clean['MSRP Quantile'].unique(), key=lambda x: float(x.split(',')[0][1:]))
df_clean['MSRP Quantile'] = pd.Categorical(df_clean['MSRP Quantile'], categories=ordered_quantiles, ordered=True)

# Regroupement
grouped = df_clean.groupby(['MSRP Quantile', 'Transmission Type']).size().reset_index(name='Nombre de véhicules')

# Création du graphique
fig = px.bar(
    grouped,
    x='MSRP Quantile',
    y='Nombre de véhicules',
    color='Transmission Type',
    text='Nombre de véhicules',
    title="Répartition des types de transmissions de véhicule par quantile de prix",
    labels={'MSRP Quantile': 'Catégorie de prix (quantiles MSRP)'}
)

# Mise en forme
fig.update_layout(
    xaxis_tickangle=-45,
    barmode='stack',
    legend_title="Type de transmission",
    yaxis_title="Nombre de véhicules",
    font=dict(size=14)
)

fig.update_traces(textposition='inside')

# Affichage dans le notebook
pyo.iplot(fig)


C:\Users\frede\AppData\Local\Temp\ipykernel_4624\33815153.py:28: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### Analyse croisée de la répartition des types de transmission en fonction du prix et du style de véhicule

#### Répartition des types de transmission selon les quantiles de prix

Le premier graphique présente la distribution des types de transmission (automatique, manuelle, automatisée manuelle, etc.) au sein de cinq quantiles de prix (MSRP). Une tendance nette se dégage :

- **Les véhicules à transmission automatique** dominent largement dans les **quantiles intermédiaires à élevés**, représentant souvent **plus de 80 %** des véhicules de ces catégories. Ce type de transmission est typique des véhicules destinés à un usage confortable ou urbain.
- **Les transmissions manuelles** sont **majoritairement présentes dans les véhicules les moins chers** (1er quantile), puis décroissent fortement à mesure que le prix augmente.
- Les **transmissions automatisées manuelles** apparaissent de manière marginale, surtout dans les véhicules haut de gamme.

Ces observations traduisent une corrélation directe entre le **niveau de prix** d’un véhicule et son **type de transmission**.

In [3]:
# Préparation des données
df_clean = df[['Transmission Type', 'Vehicle Style']].dropna()
df_clean = df_clean[df_clean['Transmission Type'] != 'UNKNOWN']

# Agrégation par transmission et style
style_trans = df_clean.groupby(['Transmission Type', 'Vehicle Style']).size().reset_index(name='Nombre de véhicules')

# Création du graphique sunburst
fig = px.sunburst(
    style_trans,
    path=['Transmission Type', 'Vehicle Style'],
    values='Nombre de véhicules',
    title="Répartition des styles de véhicule selon le type de transmission"
)

# Affichage du pourcentage et étiquettes
fig.update_traces(
    textinfo='label+percent entry',
    insidetextorientation='radial'
)

# Taille du graphique
fig.update_layout(
    font=dict(size=14),
    width=900,
    height=700
)

# Affichage dans VS Code
pyo.iplot(fig)


#### Répartition des styles de véhicule selon le type de transmission

Le second graphique, un diagramme sunburst, permet de visualiser la répartition des **styles de véhicule** en fonction du **type de transmission** :

- Les **transmissions automatiques** concernent principalement des véhicules de type **SUV** et **berlines (Sedan)**, c’est-à-dire des modèles généralement plus spacieux, orientés vers le confort et souvent utilisés dans des contextes urbains ou familiaux.
- Les **transmissions manuelles** sont davantage associées à des **coupés**, **hatchbacks**, ou encore des **pickups**. Ces styles sont souvent liés à des usages plus économiques, sportifs ou professionnels.
- Les transmissions automatisées et directes (DIRECT_DRIVE) sont très minoritaires, ce qui les rend peu représentatives pour l’analyse.

#### Corrélation entre les deux graphiques

Les deux visualisations convergent vers une **relation structurelle entre le prix, le type de transmission et le style du véhicule** :

| Type de transmission | Styles dominants               | Gamme de prix associée  |
|----------------------|--------------------------------|-------------------------|
| Automatique          | SUV, Sedan                     | Moyenne à élevée        |
| Manuelle             | Coupé, Hatchback, Pickup       | Basse à moyenne         |
| Semi-automatique     | Divers véhicules haut de gamme | Élevée                  |

Cette corrélation permet de tirer des conclusions utiles à la fois en matière de compréhension du marché et dans une logique de conseil à l’achat.

#### En tant que vendeur:

Dans une optique de vente ou de recommandation:

- Pour un **client recherchant le confort ou un usage familial**, les véhicules **automatiques de type SUV ou berline** seront les plus adaptés.
- Pour un **client au budget restreint**, un **véhicule manuel** de type **compact** ou **coupé** sera plus accessible.
- Pour un **profil professionnel ou rural**, le **pickup manuel** reste un bon compromis entre robustesse et maîtrise du budget.

On peut **établir un profil type des acheteurs selon leur budget, leur usage et leurs attentes en matière de confort ou performance.**


In [5]:
# Suppression des lignes avec valeurs manquantes
df_clean = df.dropna(subset=['Engine HP', 'highway MPG', 'city mpg', 'Engine Cylinders']).copy()

# Calcul de la consommation moyenne
df_clean['Average MPG'] = (df_clean['highway MPG'] + df_clean['city mpg']) / 2

# Filtrage : suppression des moteurs à 0 cylindre (valeurs aberrantes)
df_clean = df_clean[df_clean['Engine Cylinders'] > 0]

# Tranches de puissance moteur
df_clean['HP Range'] = pd.qcut(df_clean['Engine HP'], q=5, labels=[
    'Très faible', 'Faible', 'Moyenne', 'Élevée', 'Très élevée'])

# Conversion des cylindres à entier
df_clean['Engine Cylinders'] = df_clean['Engine Cylinders'].round().astype('Int64')

# Agrégation
df_filtered = df_clean.groupby(['HP Range', 'Engine Cylinders']).agg(
    avg_mpg=('Average MPG', 'mean'),
    std_mpg=('Average MPG', 'std')
).reset_index()

# Graphique interactif
fig = px.line(
    df_filtered,
    x='HP Range',
    y='avg_mpg',
    error_y='std_mpg',
    color='Engine Cylinders',
    markers=True,
    title="Consommation moyenne par puissance moteur",
    labels={
        "HP Range": "Type de puissance moteur",
        "avg_mpg": "Consommation moyenne (MPG)",
        "Engine Cylinders": "Nombre de cylindres"
    }
)

fig.update_layout(
    legend_title_text='Nb Cylindres',
    yaxis_title="Consommation moyenne (MPG)",
    xaxis_title="Type de puissance moteur"
)

fig.show()


C:\Users\frede\AppData\Local\Temp\ipykernel_4624\1869283908.py:18: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




## Analyse croisée de la consommation moyenne selon la puissance moteur et le nombre de cylindres

Ce graphique interactif illustre la consommation moyenne de carburant (exprimée en miles par gallon – MPG) en fonction de deux caractéristiques mécaniques clés : la puissance du moteur et le nombre de cylindres. L’objectif est de mettre en évidence les liens entre performance moteur et rendement énergétique.

### Distribution générale : plus de puissance, moins de sobriété

On observe une **diminution progressive de la consommation moyenne (MPG) à mesure que la puissance augmente**. Cela confirme l’intuition selon laquelle les véhicules les plus puissants, quelle que soit leur configuration mécanique, sont aussi les moins économes en carburant. Cette tendance peut s’expliquer par plusieurs facteurs :

- Une **augmentation de la cylindrée et du couple** nécessite plus d’énergie, donc plus de carburant.
- Les véhicules puissants sont souvent plus **lourds, rapides ou orientés performance**, ce qui augmente naturellement la consommation.
- Une **sollicitation plus importante du moteur** dans les plages de puissance supérieures induit des pertes énergétiques.

### L'influence du nombre de cylindres

En croisant la puissance avec le nombre de cylindres, plusieurs observations se dégagent :

- Les **moteurs 3 et 4 cylindres** restent globalement les plus sobres, même dans les tranches moyennes de puissance.
- À partir de **6 cylindres**, la consommation moyenne augmente sensiblement et suit une évolution relativement stable selon la puissance.
- Les moteurs à **10, 12, voire 16 cylindres** présentent une consommation très élevée, cohérente avec leur usage dans des véhicules de luxe, sportifs ou utilitaires haut de gamme.

Cette répartition démontre que le **nombre de cylindres est un indicateur fiable de rendement énergétique**, en complément de la puissance brute du moteur.

### Hypothèses explicatives et interprétations

Plusieurs hypothèses permettent de compléter l’interprétation de ces résultats :

- **Optimisation technologique** : certains moteurs 4 cylindres très performants affichent une consommation contenue grâce aux innovations (turbo, hybridation, désactivation de cylindres, etc.).
- **Catégories des véhicules** : les véhicules à fort nombre de cylindres appartiennent souvent à des catégories plus énergivores (SUV, sportives, utilitaires).
- **Évolution réglementaire** : des motorisations avec une réduction de cylindrée apparaissent dans les tranches de puissance élevées, pour concilier performance et sobriété.

### Conclusion

Ce graphique montre que la consommation moyenne des véhicules est **étroitement corrélée à la puissance moteur et au nombre de cylindres**. Si la tendance générale confirme qu’un moteur plus puissant consomme davantage, des nuances apparaissent selon les technologies utilisées et les segments de véhicules. Cette analyse permet d'évaluer les compromis entre performance et efficacité énergétique pour les voitures que nous souhaitons vendre. Il serait interessant de faire une étude comparative des prix des véhicules en fonction de son moteur et d'observer quels sont les catégories de voitures qui se démarquent.



In [6]:
# Nettoyage et préparation
df_clean = df.dropna(subset=['Engine HP', 'Market Category', 'MSRP']).copy()
df_clean['HP Range'] = pd.qcut(df_clean['Engine HP'], q=5, labels=[
    'Très faible', 'Faible', 'Moyenne', 'Élevée', 'Très élevée'])
df_clean['Market Category'] = df_clean['Market Category'].str.split(',')
df_exploded = df_clean.explode('Market Category')

# Calcul du prix moyen
df_price = df_exploded.groupby(['HP Range', 'Market Category'])['MSRP'].mean().reset_index()
df_price['MSRP'] = df_price['MSRP'].round(0)

# Nombre de véhicules
vehicle_counts = df_exploded.groupby(['HP Range', 'Market Category']).size().reset_index(name='Nombre de véhicules')
df_price_annotated = df_price.merge(vehicle_counts, on=['HP Range', 'Market Category'])

# Création du graphique avec annotations visibles
fig = go.Figure()
categories = df_price_annotated['Market Category'].unique()
hp_order = ['Très faible', 'Faible', 'Moyenne', 'Élevée', 'Très élevée']

for category in categories:
    df_cat = df_price_annotated[df_price_annotated['Market Category'] == category]
    fig.add_trace(go.Bar(
        x=df_cat['HP Range'],
        y=df_cat['MSRP'],
        name=category,
        text=df_cat['Nombre de véhicules'],
        textposition='outside',  # texte visible au-dessus des barres
        hovertemplate='<b>%{x}</b><br>Catégorie: ' + category + '<br>Prix moyen: $%{y}<br>Nombre de véhicules: %{text}',
    ))

# Mise en forme
fig.update_layout(
    barmode='group',
    title="Prix moyen (MSRP) par catégorie de voiture et puissance moteur (avec nombre de véhicules)",
    xaxis_title="Type de puissance moteur",
    yaxis_title="Prix moyen des véhicules ($)",
    legend_title="Catégorie de véhicule",
    xaxis={'categoryorder': 'array', 'categoryarray': hp_order}
)

fig.show()



C:\Users\frede\AppData\Local\Temp\ipykernel_4624\2785717658.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\frede\AppData\Local\Temp\ipykernel_4624\2785717658.py:13: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Analyse croisée : prix moyen des véhicules par catégorie de voitures et puissance moteur

Le graphique représente le **prix moyen (`MSRP`)** des véhicules selon leur **catégorie de voitures** et leur **Type de puissance moteur**, tout en indiquant le **nombre de véhicules** disponibles dans chaque groupe. Cette visualisation vise à évaluer dans quelle mesure il est possible pour un client d’opter pour un véhicule **puissant** tout en maîtrisant son **budget**, malgré une consommation potentiellement plus élevée.

### Plus de puissance veut dire plus cher ?

Une tendance générale se dégage : **le prix augmente avec la puissance moteur**. Toutefois, cette relation n’est **pas systématique** :

- La catégorie **"Luxury"** s’étend sur toutes les tranches de puissance, avec des prix relativement **stables** autour de 60 000 $ à 90 000 $.
- Les catégories **"High-Performance"** et **"Exotic"** affichent des **prix très élevés** dès que la puissance atteint les tranches "Élevée" et "Très élevée", culminant jusqu’à 250 000 $ pour les véhicules "Exotic".
- Certaines catégories comme **"Crossover"** ou **"Performance"** présentent une **croissance plus modérée** du prix malgré l’augmentation de la puissance.

### Des compromis accessibles

Certaines combinaisons permettent au client de viser une **voiture puissante à un budget modéré** :

- Les véhicules **"Performance"**, présents dans les tranches "Élevée" et "Très élevée", affichent un **prix moyen de 40 000 à 60 000 $**, soit bien en dessous des véhicules "Luxury" ou "Exotic" de puissance équivalente.
- La catégorie **"Crossover"**, bien que typiquement associée à des usages utilitaires, reste présente dans toutes les tranches de puissance, avec un **prix relativement stable (~40 000 $)**.

Ces segments offrent des **alternatives intéressantes** pour un acheteur souhaitant privilégier la puissance sans entrer dans les sphères du très haut de gamme.

### Hypothèse de rentabilité pour faire une concession sur la consommation

Il peut être **pertinent** de faire une **concession sur la consommation**, si le **prix d’acquisition reste abordable**. En effet :

- Les véhicules **puissants mais abordables** en prix moyen comme certains "Performance" ou "Flex Fuel" peuvent être **rentables à long terme**, surtout si la consommation est compensée par un usage limité et maîtrisé. A **court terme", si l'utilisation du véhicule est de courtes durées.
- À l’inverse, les véhicules très puissants dans les catégories **"Exotic"** ou **"High-Performance"**, en plus d’une consommation élevée, exigent un **investissement initial considérable**, réduisant fortement leur rentabilité dans une perspective classique.

### Équilibre entre performance et budget

Ce graphique met en évidence un **équilibre atteignable** entre puissance moteur et prix d’achat pour certaines catégories :

- **"Performance"**, **"Crossover"** ou même certains véhicules **"Luxury"** de gamme intermédiaire offrent des compromis intéressants.
- Ces segments permettent d’accéder à des véhicules **puissants mais non excessifs en prix**, pour un public recherchant performance et budget maîtrisé.

### Conclusion

> Il est possible pour un client d’acheter une **voiture puissante** en acceptant une **consommation plus élevée**, **à condition de bien choisir la catégorie de véhicule**.  
> L’analyse montre que certaines **catégories de marché offrent une puissance élevée à un prix modéré**, ouvrant la voie à une rentabilité acceptable malgré la consommation.

Ainsi, l’acheteur rationnel dispose d’une **marge de manœuvre** pour arbitrer entre puissance, consommation et coût, en fonction de ses priorités comme la performance, budget, confort ou image.
